# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [1]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path): #Reads files?
    for root, dirnames, filenames in os.walk(path): #os.walk gets every file on iteration
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n': #Skips header
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification): #Gets path and classification, appends rows
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []}) #Creating 2 columns
pth=os.getcwd() #Gets current directory
data = data.append(dataFrameFromDirectory(f"{pth}/emails/spam", 'spam')) 
data = data.append(dataFrameFromDirectory(f"{pth}/emails/ham", 'ham'))


Let's have a look at that DataFrame:

In [2]:
data.head()

,message,class
D:\DA\MLCourse/emails/spam\00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam
D:\DA\MLCourse/emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
D:\DA\MLCourse/emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
D:\DA\MLCourse/emails/spam\00004.eac8de8d759b7e74154f142194282724,##############################################...,spam
D:\DA\MLCourse/emails/spam\00005.57696a39d7d84318ce497886896bf90d,I thought you might like these:\n\n1) Slim Dow...,spam


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [9]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values) #From our messages, Counts how many times each word occurs
#This splits message into a list of words and counts it.

classifier = MultinomialNB()
targets = data['class'].values #Gets all values from CLASS column

classifier.fit(counts, targets) #x.fit(train,target)

MultinomialNB()

Let's try it out:

In [16]:
examples = ['Free Viagra now!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples) #Splits message into a list of words
predictions = classifier.predict(example_counts) #MultinomiaNB().predict(Our thingy)
predictions

array(['spam', 'ham'], dtype='<U4')

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.